In [9]:
import numpy as np
import tensorflow as tf
import tensornets as nets
import os
from PIL import Image
from tqdm import tqdm, tqdm_notebook
import pickle
import math

### Implementation Guide:  
- [X] Read In Images  
- [ ] Create Labels
- [ ] Split data into training and validation- 
- [ ] Load in Model
- [ ] Change last layer of model
- [ ] Manipulate images to fit model
- [ ] Train Model with new data
- [ ] Validate Perfomance
- [ ] Save Model
- [ ] Write function to classify 1 image

### Reading Images From File

In [6]:
classes = ["Can", "Cookies", "Eggs", "Empty", "Fruit"]
dataDict = {classID: [] for classID in classes}
img_dir = "./Classes/"
for classID in tqdm_notebook(classes):
    path = img_dir + classID
    images = os.listdir(path)
    #print(len(images), classID)
    for image in tqdm_notebook(images):
        pic = Image.open(os.path.join(path,image))
        pix = np.array(pic.getdata()).reshape(pic.size[0], pic.size[1], 3)
        dataDict[classID] = dataDict[classID] + [pix]
    

### Splitting into train and validate

In [14]:
def one_hot_encode(classes):
    encoded = np.zeros((len(x), 5))
    
    for idx, val in enumerate(x):
        encoded[idx][val] = 1
        
    return encoded


In [30]:
def train_validate_split(validfrac = .1):
    valid_features = []
    valid_labels = []
    train_features = []
    train_labels = []
    for classID in tqdm(classes):
        num_images = len(dataDict[classID])
        validIdx = math.ceil(num_images*validfrac)
        valid_features.extend(dataDict[classID][:validIdx])
        valid_labels.extend([classID]*validIdx)
        train_features.extend(dataDict[classID][validIdx:])
        train_labels.extend([classID]*(num_images-validIdx))
    return (valid_features, valid_labels, train_features, train_labels)

(valid_features, valid_labels, train_features, train_labels) = train_validate_split()
print(len(valid_features), len(valid_labels))
print(len(train_features), len(train_labels))

        
        
        
    
    

100%|██████████| 5/5 [00:00<00:00, 26613.60it/s]

29 29
240 240


## INPUT and OUTPUT tensors

In [ ]:
x = tf.placeholder(tf.float32, shape=(None, 224, 224, 3), name='input_x')
y = tf.placeholder(tf.float32, shape=(None, 10), name='output_y')



In [ ]:
### HYPER-PARAMETERS
learning_rate = 0.00001
epochs = 16
batch_size = 16

In [ ]:
logits = nets.VGG19(x, is_training=True, classes=5)
model = tf.identity(logits,name='logits')
loss = tf.losses.softmax_cross_entropy(y,logits)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [ ]:
logits.print_outputs()

References
https://github.com/deep-diver/CIFAR10-VGG19-Tensorflow/blob/master/CIFAR10-transfer-learning-tensornets.ipynb  
https://towardsdatascience.com/transfer-learning-in-tensorflow-9e4f7eae3bb4